# Setup

In [1]:
# Install module needed for the project
!pip install geoalchemy2
!pip install geopandas
!pip install psycopg2-binary

In [2]:
# Import statements needed for the project
import json
import pathlib
import urllib.parse
import os

import geoalchemy2 as gdb
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import requests
import shapely
import sqlalchemy as db

from sqlalchemy.orm import declarative_base
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine, Column, String, Integer, Date, Float, MetaData, Table, create_engine
import psycopg2

# Part 1: Data Preprocessing

## 1.1) NYC Open Data

In [3]:
## Store URL and application token
url_complaint = "https://data.cityofnewyork.us/resource/erm2-nwe9.geojson"
url_tree = "https://data.cityofnewyork.us/resource/5rq2-4hqu.geojson"
app_token = "YgXUUmVq41Z9433qxe5qpLOMG"

# Define date range
start_date = "2022-10-01"
end_date = "2023-09-30"

## Define parameter
params_complaint = {"$$app_token": app_token,
                    "$select": "incident_zip,created_date,location",
                    "$where": f"created_date between '{start_date}T00:00:00' and '{end_date}T23:59:59'",
                    "$limit": 1000}

params_tree = {"$$app_token": app_token,
               "$select": "zipcode,created_at, spc_common,health,status",
               "$limit": 1000}

## Import NYC Open Data
raw_data_complaint = requests.get(url_complaint, params=params_complaint).json()
raw_data_tree = requests.get(url_tree, params=params_tree).json()

# Convert JSON to DataFrame
gdf_complaint = gpd.GeoDataFrame.from_features(raw_data_complaint['features'])
gdf_tree = gpd.GeoDataFrame.from_features(raw_data_tree['features'])

In [4]:
gdf_complaint

,geometry,incident_zip,created_date
0,POINT (-73.95918 40.65567),11226,2023-09-30T23:59:58.000
1,POINT (-73.78752 40.76676),11361,2023-09-30T23:59:38.000
2,POINT (-73.98487 40.71950),10002,2023-09-30T23:59:35.000
3,POINT (-73.79729 40.68750),11435,2023-09-30T23:59:34.000
4,POINT (-73.95795 40.65220),11226,2023-09-30T23:59:28.000
...,...,...,...
995,POINT (-73.98164 40.76330),10019,2023-09-30T22:35:17.000
996,POINT (-73.89735 40.75889),11370,2023-09-30T22:35:17.000
997,POINT (-73.95548 40.80428),10026,2023-09-30T22:35:07.000
998,POINT (-73.91600 40.81874),10455,2023-09-30T22:35:04.000


In [5]:
gdf_tree

,geometry,health,zipcode,spc_common,status,created_at
0,None,Fair,11375,red maple,Alive,08/27/2015
1,None,Fair,11357,pin oak,Alive,09/03/2015
2,None,Good,11211,honeylocust,Alive,09/05/2015
3,None,Good,11211,honeylocust,Alive,09/05/2015
4,None,Good,11215,American linden,Alive,08/30/2015
...,...,...,...,...,...,...
995,None,None,10306,None,Dead,08/31/2015
996,None,Poor,11231,Norway maple,Alive,08/26/2015
997,None,Good,11230,Norway maple,Alive,09/01/2015
998,None,Good,10304,Norway maple,Alive,08/26/2015


## 1.2) Geometric boundary data 

In [6]:
### Define the path to the SHP file (and related files)
# Path for Irene 'Documents', 'GitHub', 'Tools-for-Analytic-Project', 'data', 'nyc_zipcodes.shp'
# Path for Sahaphon r'C:\Users\USER\Documents\GitHub\Tools-for-Analytic-Project\data\nyc_zipcodes.shp'
Geom_file_path = os.path.join(os.getcwd(),'data', 'nyc_zipcodes.shp')

### Load the shapefile using geopandas
gdf_zipcode = gpd.read_file(Geom_file_path)
gdf_zipcode


,ZIPCODE,BLDGZIP,PO_NAME,POPULATION,AREA,STATE,COUNTY,ST_FIPS,CTY_FIPS,URL,SHAPE_AREA,SHAPE_LEN,geometry
0,11436,0,Jamaica,18681.0,2.269930e+07,NY,Queens,36,081,http://www.usps.com/,0.0,0.0,"POLYGON ((1038098.252 188138.380, 1038141.936 ..."
1,11213,0,Brooklyn,62426.0,2.963100e+07,NY,Kings,36,047,http://www.usps.com/,0.0,0.0,"POLYGON ((1001613.713 186926.440, 1002314.243 ..."
2,11212,0,Brooklyn,83866.0,4.197210e+07,NY,Kings,36,047,http://www.usps.com/,0.0,0.0,"POLYGON ((1011174.276 183696.338, 1011373.584 ..."
3,11225,0,Brooklyn,56527.0,2.369863e+07,NY,Kings,36,047,http://www.usps.com/,0.0,0.0,"POLYGON ((995908.365 183617.613, 996522.848 18..."
4,11218,0,Brooklyn,72280.0,3.686880e+07,NY,Kings,36,047,http://www.usps.com/,0.0,0.0,"POLYGON ((991997.113 176307.496, 992042.798 17..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
258,10310,0,Staten Island,25003.0,5.346328e+07,NY,Richmond,36,085,http://www.usps.com/,0.0,0.0,"POLYGON ((950767.507 172848.969, 950787.510 17..."
259,11693,0,Far Rockaway,11052.0,3.497516e+06,NY,Kings,36,047,http://www.usps.com/,0.0,0.0,"POLYGON ((1028453.995 167153.410, 1027813.010 ..."
260,11249,0,Brooklyn,28481.0,1.777221e+07,NY,Kings,36,047,http://www.usps.com/,0.0,0.0,"POLYGON ((995877.318 203206.075, 995968.511 20..."
261,10162,1,New York,0.0,2.103489e+04,NY,New York,36,061,http://www.usps.com/,0.0,0.0,"POLYGON ((997731.761 219560.922, 997641.948 21..."


In [7]:
# Reproject gdf to EPSG:4326
gdf_zipcode = gdf.to_crs(epsg=4326)

# Check the new CRS to confirm the change
print(gdf_zipcode.crs)

NameError: name 'gdf' is not defined

## 1.3) Rent data 

In [8]:
### Define selected column (date)
rent_selecte_column = ['2023-08-31', '2023-01-31']

### Define the path to the CSV file
# Path for Irene 'data', 'zillow_rent_data.csv'
# Path for Sahaphon r'C:\Users\USER\Documents\GitHub\Tools-for-Analytic-Project\data\zillow_rent_data.csv'
rent_file_path = os.path.join(os.getcwd(),'data', 'zillow_rent_data.csv')

### Convert CSV to DataFrame
df = pd.read_csv(rent_file_path,usecols=rent_selecte_column)
df

,2023-01-31,2023-08-31
0,2027.438438,2053.486247
1,1738.217986,1795.384582
2,1706.900064,1757.602011
3,1458.063897,1488.180414
4,2895.699421,3064.476503
...,...,...
6717,3509.210744,3310.302151
6718,NaN,2639.938102
6719,NaN,NaN
6720,2169.143026,2383.185013


## Part 2: Storing Data

In [9]:
## Create a new database in PostgreSQL
!createdb group48project

createdb: error: database creation failed: ERROR:  database "group48project" already exists


In [10]:
## Turn on the PostGIS extension
!psql --dbname group48project -c 'CREATE EXTENSION postgis;'

ERROR:  extension "postgis" already exists


In [11]:
## Create Schema File (schema.sql):

ZIPCODE_SCHEMA = """
CREATE TABLE IF NOT EXISTS nyc_zip_codes (
    zip_code VARCHAR(10) PRIMARY KEY,
    borough VARCHAR(255),
    neighborhood VARCHAR(255)
);
"""

NYC_311_SCHEMA = """
CREATE TABLE IF NOT EXISTS complaints_311 (
    complaint_id SERIAL PRIMARY KEY,
    geometry GEOMETRY(Point),
    incident_zip VARCHAR(10),
    created_date TIMESTAMP
);
"""

NYC_TREE_SCHEMA = """
CREATE TABLE IF NOT EXISTS trees (
    tree_id SERIAL PRIMARY KEY,
    zipcode VARCHAR(10),
    created_at DATE,
    spc_common VARCHAR(255),
    health VARCHAR(50),
    status VARCHAR(50)
);
"""

ZILLOW_SCHEMA = """
CREATE TABLE IF NOT EXISTS historical_rents (
    id SERIAL PRIMARY KEY,
    zip_code VARCHAR(10),
    date DATE,
    average_rent NUMERIC
);  
"""

In [12]:
# create that required schema.sql file
with open('schema.sql', 'w') as file:
    file.write(ZIPCODE_SCHEMA)
    file.write('\n')
    file.write(NYC_311_SCHEMA)
    file.write('\n')
    file.write(NYC_TREE_SCHEMA)
    file.write('\n')
    file.write(ZILLOW_SCHEMA)

SyntaxError: invalid syntax (2076906397.py, line 1)

In [14]:
from sqlalchemy import create_engine
from geoalchemy2 import Geometry
import geopandas as gpd

# Database credentials
dbname = 'group48project'
user = 'postgres'
host = 'localhost'
port = '5432'

# Create an SQLAlchemy engine
engine = create_engine(f'postgresql+psycopg2://{user}@{host}:{port}/{dbname}')

# Define the table name
table_name = 'complaints_311'

# Use the known SRID since we have normalized the data
srid = 4326

# Assuming 'geometry' is the name of the geometry column in your GeoDataFrame
gdf_complaint.to_sql(table_name, engine, if_exists='replace', index=False, 
                     dtype={'geometry': Geometry('POINT', srid=srid)}, method='multi')

# Close the engine
engine.dispose()


ProgrammingError: (psycopg2.ProgrammingError) can't adapt type 'Point'
[SQL: INSERT INTO complaints_311 (geometry, incident_zip, created_date) VALUES (ST_GeomFromEWKT(%(geometry_m0)s), %(incident_zip_m0)s, %(created_date_m0)s), (ST_GeomFromEWKT(%(geometry_m1)s), %(incident_zip_m1)s, %(created_date_m1)s), (ST_GeomFromEWKT(%(geometry_m2)s), %(incident_zip_m2)s, %(created_date_m2)s), (ST_GeomFromEWKT(%(geometry_m3)s), %(incident_zip_m3)s, %(created_date_m3)s), (ST_GeomFromEWKT(%(geometry_m4)s), %(incident_zip_m4)s, %(created_date_m4)s), (ST_GeomFromEWKT(%(geometry_m5)s), %(incident_zip_m5)s, %(created_date_m5)s), (ST_GeomFromEWKT(%(geometry_m6)s), %(incident_zip_m6)s, %(created_date_m6)s), (ST_GeomFromEWKT(%(geometry_m7)s), %(incident_zip_m7)s, %(created_date_m7)s), (ST_GeomFromEWKT(%(geometry_m8)s), %(incident_zip_m8)s, %(created_date_m8)s), (ST_GeomFromEWKT(%(geometry_m9)s), %(incident_zip_m9)s, %(created_date_m9)s), (ST_GeomFromEWKT(%(geometry_m10)s), %(incident_zip_m10)s, %(created_da